## Set up

In [ ]:
from rdflib import ConjunctiveGraph
from SPARQLWrapper import SPARQLWrapper, JSON
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import numpy as np


import pandas as pd

def remoteQuery(query, endpoint):
    endpoint.setQuery(query)
    try:
        result = endpoint.queryAndConvert()
        pd.set_option("display.max_rows",None,"display.max_colwidth",6000,"display.width",6000,)
        df = pd.DataFrame(result['results']['bindings'])
        df = df.applymap(lambda x: x['value'])
        return df
        #return (result['results']['bindings'])
    except Exception as e:
        print(e)


In [ ]:
ep_biotools = SPARQLWrapper("http://localhost:7200/repositories/Project25")
ep_biotools.setReturnFormat(JSON)

## How many EDAM data are there in EDAM?

In [ ]:
q= """
PREFIX edam:<http://edamontology.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>

PREFIX sc: <http://schema.org/>
SELECT (COUNT(DISTINCT ?data) as ?count) WHERE {
    ?data rdfs:subClassOf <http://edamontology.org/data_0006>
}
"""
print(f"Total number of EDAM data: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
remoteQuery(query=q, endpoint=ep_biotools)

## How many EDAM data are used to annotate bio.tools?

In [ ]:
q= """
PREFIX edam:<http://edamontology.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>

PREFIX sc: <http://schema.org/>
SELECT (COUNT(DISTINCT ?data) as ?count) WHERE {
    {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:input [sc:additionalType ?data].        
    } UNION {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:output [sc:additionalType ?data].        
    }
}
"""
print(f"Total number of EDAM data used to annotate bio.tools: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
remoteQuery(query=q, endpoint=ep_biotools)

## How many bio.tools entries are annotated with EDAM data?

In [ ]:
q= """
PREFIX edam:<http://edamontology.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>

PREFIX sc: <http://schema.org/>
SELECT (COUNT(DISTINCT ?biotools_id) as ?count) WHERE {
    {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:input [sc:additionalType ?data].        
    } UNION {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:output [sc:additionalType ?data].        
    }
}
"""
print(f"Total number of bio.tools entries used to annotate EDAM data: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
remoteQuery(query=q, endpoint=ep_biotools)

## Which EDAM data are used for bio.tools annotation?

In [ ]:
q= """
PREFIX edam:<http://edamontology.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>

PREFIX sc: <http://schema.org/>
SELECT DISTINCT ?data WHERE {
    {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:input [sc:additionalType ?data].        
    } UNION {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:output [sc:additionalType ?data].        
    }
}

ORDER BY ASC(?data)
"""
print(f"EDAM data used to annotate bio.tools:")
remoteQuery(query=q, endpoint=ep_biotools)

In [ ]:
q= """
PREFIX edam:<http://edamontology.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>

PREFIX sc: <http://schema.org/>
SELECT (COUNT(DISTINCT ?data) as ?count) WHERE {
    {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:input [sc:additionalType ?data].
   
    } UNION {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:output [sc:additionalType ?data].        
    }
}
"""
print(f"Total number of EDAM data used to annotate bio.tools: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
remoteQuery(query=q, endpoint=ep_biotools)

## Which deprecated EDAM data are used for bio.tools annotation?

In [ ]:
q= """
PREFIX edam:<http://edamontology.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>

PREFIX sc: <http://schema.org/>
SELECT DISTINCT ?data WHERE {
    {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:input [sc:additionalType ?data].
   ?data rdfs:subClassOf <http://www.w3.org/2002/07/owl#DeprecatedClass> . 
    } UNION {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:output [sc:additionalType ?data].        
   ?data rdfs:subClassOf <http://www.w3.org/2002/07/owl#DeprecatedClass> . 
    }
}

ORDER BY ASC(?data)
"""
print(f"EDAM data used to annotate bio.tools:")
remoteQuery(query=q, endpoint=ep_biotools)